In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# CLUSTERING APRIMORADO - CAPTURANDO MÚLTIPLOS PADRÕES ALIMENTARES
# Melhorias: Tratamento de dados binários, múltiplos métodos, validação robusta

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, SpectralClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.feature_selection import VarianceThreshold
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from scipy.spatial.distance import pdist
from kneed import KneeLocator
import warnings
warnings.filterwarnings('ignore')

print("🍎 CLUSTERING APRIMORADO - ANÁLISE DE PADRÕES ALIMENTARES INFANTIS")
print("=" * 80)
print("🎯 Objetivo: Identificar múltiplos grupos com padrões alimentares distintos")
print("📊 Melhorias: Tratamento especial para dados binários, múltiplos algoritmos")
print()

# ====================================================================
# 1. CARREGAR E PREPARAR DADOS
# ====================================================================

print("📊 1. CARREGANDO E PREPARANDO DATASET")
print("-" * 40)

df = pd.read_csv('/Users/marcelosilva/Desktop/clustering(0-4)/3-E-Aval/DSFinal.csv')

print(f"✅ Dataset original: {df.shape[0]:,} crianças × {df.shape[1]} variáveis")

# Remover variáveis de frequência e preparar dados
frequency_vars = ['e13_fruta_vezes', 'e17_sal_vezes']
# Verificar quais colunas realmente existem
cols_to_remove = [col for col in frequency_vars if col in df.columns]
if cols_to_remove:
    df_filtered = df.drop(columns=cols_to_remove)
else:
    df_filtered = df.copy()

# Verificar e remover colunas string se existirem
string_cols = ['e18_oferecida', 'e21a_pao']
existing_string_cols = [col for col in string_cols if col in df_filtered.columns]
if existing_string_cols:
    df_clean = df_filtered.drop(columns=existing_string_cols)
else:
    df_clean = df_filtered.copy()

X = df_clean.drop('id_anon', axis=1)
ids = df_clean['id_anon']

print(f"✅ Dataset limpo: {X.shape[0]:,} crianças × {X.shape[1]} variáveis")

# Análise da natureza dos dados
binary_count = sum((X[col].nunique() == 2) for col in X.columns)
print(f"📊 Variáveis binárias: {binary_count}/{X.shape[1]} ({binary_count/X.shape[1]*100:.1f}%)")

# Verificar distribuição dos dados
print("\n🔍 Análise da distribuição dos dados:")
prevalence = X.mean()
print(f"   Alimentos com prevalência > 80%: {sum(prevalence > 0.8)}")
print(f"   Alimentos com prevalência < 20%: {sum(prevalence < 0.2)}")
print(f"   Alimentos com prevalência 20-80%: {sum((prevalence >= 0.2) & (prevalence <= 0.8))}")

# Listar colunas disponíveis para debugging
print(f"\n📋 Primeiras 10 variáveis disponíveis: {list(X.columns[:10])}")

# ====================================================================
# 2. TRANSFORMAÇÃO INTELIGENTE DOS DADOS
# ====================================================================

print("\n⚙️ 2. TRANSFORMAÇÃO INTELIGENTE DOS DADOS")
print("-" * 40)

# Criar diferentes representações dos dados
transformations = {}

# 1. Dados originais (binários)
transformations['Original'] = X.values

# 2. PCA para capturar variância principal
pca = PCA(n_components=0.95, random_state=42)  # Manter 95% da variância
X_pca = pca.fit_transform(X)
transformations['PCA'] = X_pca
print(f"✅ PCA: {X_pca.shape[1]} componentes mantendo 95% da variância")

# 3. Criação de perfis agregados MELHORADOS
print("📊 Criando perfis agregados avançados...")

# Identificar grupos baseados no ENANI
X_perfis = pd.DataFrame()

# IDADE 0-6 MESES (aleitamento materno predominante)
if 'e01_leite_peito' in X.columns:
    X_perfis['amamentacao'] = X['e01_leite_peito']
    
# SUBSTITUTOS DO LEITE MATERNO
leites_substitutos = ['e06_leite_vaca_po', 'e07_leite_vaca_liquido', 
                     'e10_formula_infantil']
leites_existentes = [col for col in leites_substitutos if col in X.columns]
if leites_existentes:
    X_perfis['leites_substitutos'] = X[leites_existentes].max(axis=1)

# INDICADOR DE INTRODUÇÃO ALIMENTAR
if 'e214a_nao_comeu' in X.columns:
    X_perfis['ja_come_solidos'] = 1 - X['e214a_nao_comeu']
    X_perfis['alimentacao_exclusiva_leite'] = X['e214a_nao_comeu']

# ALIMENTOS BÁSICOS BRASILEIROS
basicos = ['e21_arroz', 'e26_feijao']
basicos_exist = [col for col in basicos if col in X.columns]
if basicos_exist:
    X_perfis['alimentos_basicos'] = X[basicos_exist].mean(axis=1)

# PROTEÍNAS
proteinas = ['e27_carne', 'e29_ovo']
proteinas_exist = [col for col in proteinas if col in X.columns]
if proteinas_exist:
    X_perfis['consumo_proteinas'] = X[proteinas_exist].max(axis=1)

# FRUTAS E VEGETAIS
frutas_veg = ['e12_fruta_inteira', 'e22_legumes', 'e23_cenoura', 'e24_couve', 'e25_verduras']
fv_exist = [col for col in frutas_veg if col in X.columns]
if fv_exist:
    X_perfis['frutas_vegetais'] = X[fv_exist].mean(axis=1)

# ULTRAPROCESSADOS (crítico no ENANI)
ultraproc = ['e30_hamburger', 'e31_salgadinhos', 'e32_suco_industrializado', 
             'e33_refrigerante', 'e35_biscoito', 'e36_bala']
ultra_exist = [col for col in ultraproc if col in X.columns]
if ultra_exist:
    X_perfis['ultraprocessados'] = X[ultra_exist].sum(axis=1) / len(ultra_exist)

# PREPARAÇÕES ESPECIAIS PARA BEBÊS
if 'e19_mingau' in X.columns:
    X_perfis['mingau'] = X['e19_mingau']

# TEXTURA DOS ALIMENTOS (indicador de desenvolvimento)
texturas = ['e181_pedacos', 'e182_amassada', 'e183_peneira', 'e184_liquidificada']
texturas_exist = [col for col in texturas if col in X.columns]
if texturas_exist:
    # Pedaços indica desenvolvimento mais avançado
    if 'e181_pedacos' in X.columns:
        X_perfis['alimentacao_solida_avancada'] = X['e181_pedacos']

# INDICADORES DE RISCO
if 'e39_mamadeira' in X.columns:
    X_perfis['uso_mamadeira'] = X['e39_mamadeira']
if 'e40_adocado' in X.columns:
    X_perfis['alimentos_adocados'] = X['e40_adocado']

# DIVERSIDADE ALIMENTAR (importante no ENANI)
X_perfis['diversidade_total'] = (X > 0).sum(axis=1) / X.shape[1]
X_perfis['n_grupos_alimentares'] = (X_perfis > 0).sum(axis=1)

# PADRÕES ESPECÍFICOS POR IDADE
# Bebês muito pequenos: só leite
X_perfis['padrao_0_6m'] = ((X_perfis.get('amamentacao', 0) > 0) | 
                           (X_perfis.get('leites_substitutos', 0) > 0)) & \
                          (X_perfis.get('ja_come_solidos', 1) == 0)

# Transição alimentar: leite + alguns sólidos
X_perfis['padrao_transicao'] = (X_perfis.get('ja_come_solidos', 0) > 0) & \
                                (X_perfis.get('diversidade_total', 0) < 0.3)

# Alimentação estabelecida: diversa
X_perfis['padrao_estabelecido'] = X_perfis.get('diversidade_total', 0) > 0.5

transformations['Perfis'] = X_perfis.values
print(f"✅ Perfis criados: {X_perfis.shape[1]} variáveis agregadas")

# 4. Combinação de múltiplas representações
X_combined = np.hstack([
    StandardScaler().fit_transform(X.values),
    StandardScaler().fit_transform(X_perfis.values)
])
transformations['Combinado'] = X_combined
print(f"✅ Representação combinada: {X_combined.shape[1]} features")

# ====================================================================
# 3. TESTE DE MÚLTIPLOS ALGORITMOS
# ====================================================================

print("\n🔬 3. TESTANDO MÚLTIPLOS ALGORITMOS DE CLUSTERING")
print("-" * 40)

# Para cada transformação, testar vários algoritmos
results_all = {}
K_range = range(4, 15)  # FORÇAR mínimo de 4 clusters

for trans_name, X_trans in transformations.items():
    print(f"\n📊 Testando com {trans_name}...")
    
    # Normalizar sempre
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X_trans)
    
    results_trans = {}
    
    # 1. K-Means com múltiplas inicializações
    print("  🔸 K-Means com inicialização k-means++...")
    for k in K_range:
        kmeans = KMeans(n_clusters=k, n_init=50, max_iter=1000, 
                       init='k-means++', random_state=42)
        labels = kmeans.fit_predict(X_scaled)
        
        # Validar clusters
        unique_labels = np.unique(labels)
        if len(unique_labels) < k:
            continue
            
        sil = silhouette_score(X_scaled, labels)
        cal = calinski_harabasz_score(X_scaled, labels)
        
        # Verificar balanceamento
        _, counts = np.unique(labels, return_counts=True)
        min_size = min(counts)
        max_size = max(counts)
        balance = min_size / max_size
        
        # Aceitar clusters menores para ter mais grupos
        if min_size >= 50:  # Reduzido de 100 para 50
            score = sil * 0.4 + (k/10) * 0.3 + balance * 0.3  # Favorece mais clusters
            results_trans[f'kmeans_{k}'] = {
                'labels': labels,
                'silhouette': sil,
                'calinski': cal,
                'n_clusters': k,
                'min_size': min_size,
                'balance': balance,
                'score': score,
                'algorithm': 'kmeans'
            }
    
    # 2. Mini-Batch K-Means (mais robusto para dados grandes)
    print("  🔸 Mini-Batch K-Means...")
    from sklearn.cluster import MiniBatchKMeans
    for k in range(5, 12):
        mbk = MiniBatchKMeans(n_clusters=k, batch_size=1000, n_init=30, random_state=42)
        labels = mbk.fit_predict(X_scaled)
        
        if len(np.unique(labels)) == k:
            sil = silhouette_score(X_scaled, labels, sample_size=5000)
            _, counts = np.unique(labels, return_counts=True)
            min_size = min(counts)
            
            if min_size >= 50:
                results_trans[f'minibatch_{k}'] = {
                    'labels': labels,
                    'silhouette': sil,
                    'n_clusters': k,
                    'min_size': min_size,
                    'algorithm': 'minibatch'
                }
    
    # 3. Hierarchical com Ward (melhor para grupos compactos)
    print("  🔸 Hierarchical Ward...")
    for k in [5, 6, 7, 8]:
        try:
            hc = AgglomerativeClustering(n_clusters=k, linkage='ward')
            labels = hc.fit_predict(X_scaled)
            
            sil = silhouette_score(X_scaled, labels, sample_size=5000)
            _, counts = np.unique(labels, return_counts=True)
            min_size = min(counts)
            
            if min_size >= 100:
                results_trans[f'hierarchical_ward_{k}'] = {
                    'labels': labels,
                    'silhouette': sil,
                    'n_clusters': k,
                    'min_size': min_size,
                    'algorithm': 'hierarchical_ward'
                }
        except:
            continue
    
    # 4. DBSCAN adaptativo (encontra clusters naturalmente)
    print("  🔸 DBSCAN adaptativo...")
    from sklearn.neighbors import NearestNeighbors
    # Encontrar eps ótimo
    nn = NearestNeighbors(n_neighbors=50)
    nn.fit(X_scaled)
    distances, _ = nn.kneighbors(X_scaled)
    distances = np.sort(distances[:, -1])
    eps_candidates = np.percentile(distances, [85, 90, 95])
    
    for eps in eps_candidates:
        dbscan = DBSCAN(eps=eps, min_samples=30)
        labels = dbscan.fit_predict(X_scaled)
        
        n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
        if 4 <= n_clusters <= 12:
            # Reassign outliers to nearest cluster
            if -1 in labels:
                from sklearn.neighbors import KNeighborsClassifier
                mask = labels != -1
                if mask.sum() > 0:
                    knn = KNeighborsClassifier(n_neighbors=5)
                    knn.fit(X_scaled[mask], labels[mask])
                    labels[~mask] = knn.predict(X_scaled[~mask])
            
            _, counts = np.unique(labels, return_counts=True)
            min_size = min(counts)
            
            if min_size >= 100 and len(counts) >= 4:
                sil = silhouette_score(X_scaled, labels, sample_size=5000)
                results_trans[f'dbscan_{eps:.2f}'] = {
                    'labels': labels,
                    'silhouette': sil,
                    'n_clusters': n_clusters,
                    'min_size': min_size,
                    'algorithm': 'dbscan'
                }
    
    results_all[trans_name] = results_trans

# ====================================================================
# 4. SELEÇÃO DO MELHOR RESULTADO
# ====================================================================

print("\n🏆 4. SELEÇÃO DO MELHOR CLUSTERING")
print("-" * 40)

# Encontrar melhor combinação
best_score = -1
best_config = None
all_configs = []

for trans_name, results in results_all.items():
    for config_name, config in results.items():
        n_clusters = config['n_clusters']
        silhouette = config.get('silhouette', 0)
        min_size = config['min_size']
        
        # Score que FORTEMENTE favorece mais clusters
        # Penaliza fortemente soluções com poucos clusters
        if n_clusters < 4:
            cluster_bonus = 0.1
        elif n_clusters < 6:
            cluster_bonus = 0.5
        else:
            cluster_bonus = 1.0 + (n_clusters - 6) * 0.1
        
        # Aceitar silhouette mais baixo em troca de mais clusters
        score = (silhouette + 0.3) * cluster_bonus * (min_size / 200)
        
        config['final_score'] = score
        config['trans_name'] = trans_name
        config['config_name'] = config_name
        all_configs.append(config)
        
        if score > best_score and n_clusters >= 5:
            best_score = score
            best_config = (trans_name, config_name, config)

# Se não encontrou solução com 5+ clusters, forçar
if best_config is None or best_config[2]['n_clusters'] < 5:
    print("⚠️ Forçando solução com mais clusters...")
    
    # Tentar K-means com k=6 ou k=7 na representação combinada
    X_scaled = StandardScaler().fit_transform(transformations['Combinado'])
    
    for k in [7, 6, 8, 5]:
        kmeans = KMeans(n_clusters=k, n_init=100, max_iter=1000, 
                       init='k-means++', random_state=42)
        labels = kmeans.fit_predict(X_scaled)
        
        _, counts = np.unique(labels, return_counts=True)
        min_size = min(counts)
        
        if min_size >= 50:  # Aceitar clusters menores
            sil = silhouette_score(X_scaled, labels, sample_size=5000)
            best_config = ('Combinado_Forcado', f'kmeans_{k}', {
                'labels': labels,
                'silhouette': sil,
                'n_clusters': k,
                'min_size': min_size,
                'algorithm': 'kmeans_forced'
            })
            break

# Mostrar top 5 configurações
print("\n📊 Top 5 configurações encontradas:")
all_configs.sort(key=lambda x: x['final_score'], reverse=True)
for i, cfg in enumerate(all_configs[:5]):
    print(f"{i+1}. {cfg['trans_name']} - {cfg['algorithm']} "
          f"(K={cfg['n_clusters']}, Sil={cfg.get('silhouette', 0):.3f}, "
          f"MinSize={cfg['min_size']}, Score={cfg['final_score']:.3f})")

if best_config:
    trans_name, config_name, config = best_config
    best_labels = config['labels']
    n_clusters = config['n_clusters']
    
    print(f"\n✅ Configuração selecionada:")
    print(f"   Transformação: {trans_name}")
    print(f"   Algoritmo: {config['algorithm']}")
    print(f"   Número de clusters: {n_clusters}")
    print(f"   Silhouette: {config.get('silhouette', 'N/A')}")
    print(f"   Menor cluster: {config['min_size']:,} crianças")
else:
    print("\n❌ Nenhuma configuração adequada encontrada!")
    print("Usando fallback com 6 clusters...")
    X_scaled = StandardScaler().fit_transform(X_perfis)
    kmeans = KMeans(n_clusters=6, n_init=50, random_state=42)
    best_labels = kmeans.fit_predict(X_scaled)
    n_clusters = 6

if best_config:
    trans_name, config_name, config = best_config
    best_labels = config['labels']
    n_clusters = config['n_clusters']
    
    print(f"✅ Melhor configuração encontrada:")
    print(f"   Transformação: {trans_name}")
    print(f"   Algoritmo: {config['algorithm']}")
    print(f"   Número de clusters: {n_clusters}")
    print(f"   Silhouette: {config['silhouette']:.3f}")
    print(f"   Menor cluster: {config['min_size']:,} crianças")
else:
    # Fallback: forçar clustering com mais grupos
    print("⚠️ Forçando clustering com 6 grupos...")
    X_scaled = StandardScaler().fit_transform(X_perfis)
    kmeans = KMeans(n_clusters=6, n_init=50, random_state=42)
    best_labels = kmeans.fit_predict(X_scaled)
    n_clusters = 6

# ====================================================================
# 5. ANÁLISE DETALHADA DOS CLUSTERS
# ====================================================================

print(f"\n📊 5. ANÁLISE DOS {n_clusters} CLUSTERS ENCONTRADOS")
print("-" * 40)

# Adicionar clusters ao dataset
df_final = df_clean.copy()
df_final['cluster'] = best_labels

# Estatísticas por cluster
for cluster in sorted(np.unique(best_labels)):
    cluster_data = df_final[df_final['cluster'] == cluster]
    n_criancas = len(cluster_data)
    pct = n_criancas / len(df_final) * 100
    
    print(f"\n🏷️ CLUSTER {cluster}")
    print(f"   Tamanho: {n_criancas:,} crianças ({pct:.1f}%)")
    
    # Calcular médias do cluster
    cluster_means = cluster_data[X.columns].mean()
    overall_means = df_final[X.columns].mean()
    
    # Top 10 alimentos
    top_10 = cluster_means.nlargest(10)
    print("\n   📈 Top 10 alimentos mais consumidos:")
    for food, value in top_10.items():
        diff = value - overall_means[food]
        diff_str = f"({diff:+.1%} vs média)" if abs(diff) > 0.1 else ""
        print(f"      • {food}: {value:.1%} {diff_str}")
    
    # Características distintivas (>20% diferença)
    diffs = cluster_means - overall_means
    distintivos = diffs[abs(diffs) > 0.2].sort_values(ascending=False)
    
    if len(distintivos) > 0:
        print("\n   ⭐ Características distintivas:")
        for food, diff in distintivos.items():
            if diff > 0:
                print(f"      ↑ {food}: {diff:+.1%} acima da média")
            else:
                print(f"      ↓ {food}: {diff:.1%} abaixo da média")
    
    # Perfil resumido
    if 'e01_leite_peito' in cluster_means.index:
        amamentacao = cluster_means['e01_leite_peito']
        solidos = 1 - cluster_means.get('e214a_nao_comeu', 0)
        ultraproc = cluster_means[['e30_hamburger', 'e31_salgadinhos', 
                                  'e32_suco_industrializado', 'e33_refrigerante', 
                                  'e35_biscoito', 'e36_bala']].mean()
        
        print(f"\n   📋 Perfil resumido:")
        print(f"      • Amamentação: {amamentacao:.1%}")
        print(f"      • Alimentos sólidos: {solidos:.1%}")
        print(f"      • Ultraprocessados: {ultraproc:.1%}")

# ====================================================================
# 6. VISUALIZAÇÃO E SALVAMENTO
# ====================================================================

print("\n💾 6. SALVANDO RESULTADOS E VISUALIZAÇÕES")
print("-" * 40)

import os
output_path = '/Users/marcelosilva/Desktop/clustering(0-4)/4-Clustering/'
os.makedirs(output_path, exist_ok=True)

# Salvar resultados principais
results_df = pd.DataFrame({
    'id_anon': ids,
    'cluster': best_labels
})
results_df.to_csv(output_path + 'clustering_multiplos_grupos.csv', index=False)
print(f"✅ Resultado principal salvo: {n_clusters} clusters")

# Dataset completo
df_final.to_csv(output_path + 'dataset_completo_clusters.csv', index=False)

# Características por cluster
cluster_profiles = df_final.groupby('cluster')[X.columns].mean()
cluster_profiles.to_csv(output_path + 'perfis_clusters.csv')

# Criar visualização
plt.figure(figsize=(15, 10))

# Subplot 1: Distribuição dos clusters
plt.subplot(2, 2, 1)
cluster_counts = df_final['cluster'].value_counts().sort_index()
plt.bar(cluster_counts.index, cluster_counts.values)
plt.title(f'Distribuição dos {n_clusters} Clusters')
plt.xlabel('Cluster')
plt.ylabel('Número de Crianças')
for i, v in enumerate(cluster_counts.values):
    plt.text(i, v + 50, f'{v:,}\n({v/len(df_final)*100:.1f}%)', 
             ha='center', va='bottom')

# Subplot 2: PCA visualization
plt.subplot(2, 2, 2)
pca_viz = PCA(n_components=2, random_state=42)
X_pca_viz = pca_viz.fit_transform(X)
scatter = plt.scatter(X_pca_viz[:, 0], X_pca_viz[:, 1], 
                     c=best_labels, cmap='tab10', alpha=0.6, s=1)
plt.colorbar(scatter)
plt.title('Visualização PCA dos Clusters')
plt.xlabel(f'PC1 ({pca_viz.explained_variance_ratio_[0]:.1%} var)')
plt.ylabel(f'PC2 ({pca_viz.explained_variance_ratio_[1]:.1%} var)')

# Subplot 3: Heatmap dos perfis
plt.subplot(2, 1, 2)
# Selecionar variáveis mais importantes
important_vars = []
for cluster in range(n_clusters):
    cluster_means = df_final[df_final['cluster'] == cluster][X.columns].mean()
    overall_means = df_final[X.columns].mean()
    diffs = abs(cluster_means - overall_means)
    important_vars.extend(diffs.nlargest(5).index.tolist())
important_vars = list(set(important_vars))[:20]  # Top 20 variáveis distintivas

heatmap_data = cluster_profiles[important_vars].T
sns.heatmap(heatmap_data, cmap='YlOrRd', cbar_kws={'label': 'Proporção de Consumo'})
plt.title('Perfil de Consumo por Cluster (Top 20 Alimentos Distintivos)')
plt.xlabel('Cluster')
plt.ylabel('Alimento')
plt.tight_layout()

plt.savefig(output_path + 'visualizacao_clusters.png', dpi=300, bbox_inches='tight')
print("✅ Visualizações salvas")

# Relatório resumido
with open(output_path + 'relatorio_clusters.txt', 'w') as f:
    f.write(f"RELATÓRIO DE CLUSTERING - PADRÕES ALIMENTARES INFANTIS\n")
    f.write("=" * 60 + "\n\n")
    f.write(f"Total de crianças: {len(df_final):,}\n")
    f.write(f"Número de clusters: {n_clusters}\n\n")
    
    for cluster in sorted(np.unique(best_labels)):
        n = len(df_final[df_final['cluster'] == cluster])
        f.write(f"\nCLUSTER {cluster}: {n:,} crianças ({n/len(df_final)*100:.1f}%)\n")
        f.write("-" * 40 + "\n")
        
        # Características principais
        cluster_means = df_final[df_final['cluster'] == cluster][X.columns].mean()
        top_5 = cluster_means.nlargest(5)
        
        f.write("Principais alimentos:\n")
        for food, value in top_5.items():
            f.write(f"  - {food}: {value:.1%}\n")

print(f"\n🎉 CLUSTERING CONCLUÍDO COM SUCESSO!")
print(f"📊 {n_clusters} clusters identificados")
print(f"📁 Resultados salvos em: {output_path}")
print("=" * 80)